In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("农业数据.xlsx")
data

,不同种类植物种粮类别的数量,粮食作物的产量(万吨）,农作物种子进口量(吨）,种业行业专利发表数,期刊论文发布量,转基因种子种植面积（万公顷）及同比情况,转基因作物种子单产,第一产业从业者人数（年底数）（万人）,农作物种植的土地面积(千公顷),种业市场的总体规模和容量
0,446,54641,10652.210,4928,1254,380,344.0,27931,158579,900
1,612,57121,11298.640,6548,1478,390,354.0,26472,160360,990
2,575,58957,11564.980,9151,1921,400,354.0,25535,162071,1039
3,386,60194,12895.210,12714,1970,420,358.5,23838,163702,1113
4,453,60710,15368.550,14166,1826,290,359.0,22372,165183,1149
5,670,62144,18265.350,20865,1482,370,365.5,21418,166829,1170
6,855,61624,19542.210,24207,1350,280,363.5,20908,166939,1221
7,662,61791,21360.190,28043,1303,280,367.0,20295,166332,1174
8,538,65789,77820.020,42028,1177,290,375.0,19515,165902,1192
9,1263,66384,72614.002,33949,1149,320,381.0,18652,165931,1400


In [3]:
lag = ['+', '+', '-', '+', '+', '+', '+', '+', '+', '+']
lag

['+', '+', '-', '+', '+', '+', '+', '+', '+', '+']

In [40]:
import pandas as pd
import numpy as np
import math


def nml(series):  # 正向指标归一化 减最小值的min-max方法
    l = []
    for i in series:
        l.append((i - series.min()) / (series.max() - series.min()))
    return pd.Series(l, name=series.name)


def nml_max(series):  # 负向指标归一化
    l = []
    for i in series:
        l.append((series.max() - i) / (series.max() - series.min()))
    return pd.Series(l, name=series.name)


def nmlzt(df, nml_nmlmax):  # 归一化函数，对正负向指标分别调用nml()和nml_max()
    dfn = pd.DataFrame()
    for i in list(range(10)):
        if (nml_nmlmax[i] == "+"):
            dfn = pd.concat([dfn, nml(data.iloc[0])], axis=1)
        else:
            dfn = pd.concat([dfn, nml_max(data.iloc[0])], axis=1)
#     print(dfn)
    # dfn为归一化的数据
    return dfn


def pij(df):  # 求信息熵公式中的p，这里直接用取值除以取值总和，而不是数量的比例
    D = df.copy()
    for i in range(D.shape[1]):  # 列
        sum = D.iloc[:, i].sum()
        for j in range(D.shape[0]):  # 行
            D.iloc[j, i] = D.iloc[j, i] / sum
            # 算pij
    return D


def entropy(series):  # 计算信息熵
    _len = len(series)

    def ln(x):
        if x > 0:
            return math.log(x)
        else:
            return 0

    s = 0
    for i in series:
        s += i * ln(i)
    return -(1 / ln(_len)) * s


def _result(dfij):  # 求e、d、w并返回
    dfn = dfij.copy()
    w = pd.DataFrame(index=dfn.columns, dtype='float64')
    l = []
    for i in dfn.columns:
        l.append(entropy(dfn[i]))
    w['熵'] = l
    w['差异性系数'] = 1 - np.array(l)
    sum = w['差异性系数'].sum()
    l = []
    for i in w['差异性系数']:
        l.append(i / sum)
    w['权重'] = l
    return w


def out(dfn, w):
    w = w['权重']
    answer = np.matmul(dfn, w)
    return answer

In [48]:
df = data
dfn = nmlzt(df, lag)  # 归一化
dfij = pij(df)  # 求p
w = _result(dfij)  # 求权重
# w.to_excel('weight_info_entropy.xlsx', sheet_name='权重')#输出结果
dfn = df.set_index(df.index, drop=True)

In [44]:
df

,不同种类植物种粮类别的数量,粮食作物的产量(万吨）,农作物种子进口量(吨）,种业行业专利发表数,期刊论文发布量,转基因种子种植面积（万公顷）及同比情况,转基因作物种子单产,第一产业从业者人数（年底数）（万人）,农作物种植的土地面积(千公顷),种业市场的总体规模和容量
0,446,54641,10652.210,4928,1254,380,344.0,27931,158579,900
1,612,57121,11298.640,6548,1478,390,354.0,26472,160360,990
2,575,58957,11564.980,9151,1921,400,354.0,25535,162071,1039
3,386,60194,12895.210,12714,1970,420,358.5,23838,163702,1113
4,453,60710,15368.550,14166,1826,290,359.0,22372,165183,1149
5,670,62144,18265.350,20865,1482,370,365.5,21418,166829,1170
6,855,61624,19542.210,24207,1350,280,363.5,20908,166939,1221
7,662,61791,21360.190,28043,1303,280,367.0,20295,166332,1174
8,538,65789,77820.020,42028,1177,290,375.0,19515,165902,1192
9,1263,66384,72614.002,33949,1149,320,381.0,18652,165931,1400


In [45]:
dfn

,0,0,0,0,0,0,0,0,0,0
0,0.000645,0.000645,0.999355,0.000645,0.000645,0.000645,0.000645,0.000645,0.000645,0.000645
1,0.343142,0.343142,0.656858,0.343142,0.343142,0.343142,0.343142,0.343142,0.343142,0.343142
2,0.065145,0.065145,0.934855,0.065145,0.065145,0.065145,0.065145,0.065145,0.065145,0.065145
3,0.028970,0.028970,0.971030,0.028970,0.028970,0.028970,0.028970,0.028970,0.028970,0.028970
4,0.005751,0.005751,0.994249,0.005751,0.005751,0.005751,0.005751,0.005751,0.005751,0.005751
5,0.000228,0.000228,0.999772,0.000228,0.000228,0.000228,0.000228,0.000228,0.000228,0.000228
6,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.174342,0.174342,0.825658,0.174342,0.174342,0.174342,0.174342,0.174342,0.174342,0.174342
8,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
9,0.003514,0.003514,0.996486,0.003514,0.003514,0.003514,0.003514,0.003514,0.003514,0.003514


In [49]:
w.to_excel("权重系数.xlsx")
w

,熵,差异性系数,权重
不同种类植物种粮类别的数量,0.970135,0.029865,0.124479
粮食作物的产量(万吨）,0.999057,0.000943,0.003930
农作物种子进口量(吨）,0.879245,0.120755,0.503316
种业行业专利发表数,0.931511,0.068489,0.285467
期刊论文发布量,0.992800,0.007200,0.030012
转基因种子种植面积（万公顷）及同比情况,0.995895,0.004105,0.017112
转基因作物种子单产,0.999725,0.000275,0.001147
第一产业从业者人数（年底数）（万人）,0.995144,0.004856,0.020241
农作物种植的土地面积(千公顷),0.999914,0.000086,0.000359
种业市场的总体规模和容量,0.996656,0.003344,0.013937


In [50]:
answer = out(dfn, w)
answer.to_excel("综合评价系数.xlsx")
answer

0      7717.740005
1      8515.231576
2      9390.762420
3     11027.828494
4     12662.381017
5     16038.005445
6     17640.610231
7     19612.626490
8     52003.020551
9     47154.163635
10    58750.052763
11    72244.896168
12    55925.179702
13    47899.039067
dtype: float64